# Similarity in Term-Frecuency Text Representation using SVD

**Prerequisites:** Skills in Sklearn, and knowledge of TfIdf Text Representation model.

## Outline

**Main Goal:**  How to applied Using Singular Value Decomposition as Dimensionality Reduction Technique on TfIdf models with Sklearn. Then introduce how to extract information from this text representation, and finally how to measure word similarity.

- TfIdf Sklearn model generation.
- Wrangling data from strings to numerical vectors.
- Applied some Sklearn, Scipy and Gensim similarity measures.
- Applied Okapi BM25 weighting model to compare with previos similarities scores.


### Import section

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
from scipy.sparse import  coo_matrix
import scipy
import time
import nltk
import ssl

### Loading Data

You could have some SSL problems, so first you must solve them:

In [5]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('gutenberg')
# check the download path for next cell

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sorice/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [7]:
#change the path to the library if you load it from local path
corpus = load_files(f'/Users/sorice/nltk_data/corpora/',categories=['gutenberg'])

### Generate Left Singular Values Unitary Matrix

Algo called in this notebook as Ug, will be used as matrix for calculations.

In [2]:
init = time.time()
gvectorizer = CountVectorizer(min_df=1, lowercase=False)
GTfMatrix = gvectorizer.fit_transform(corpus.data)
coGTfMatrix = coo_matrix(GTfMatrix.T)
coGTfMatrix.data = coGTfMatrix.data*1.0 #a necessary step because without it doesn't work
#k must be between 1 and min(coGTfMatrix.shape)
print(coGTfMatrix.shape)
Ug, Sg, Vg = scipy.sparse.linalg.svds(coGTfMatrix, k=20)
end = time.time()-init
print('Total time:', end)

(63341, 21)
Total time: 1.9103684425354004


In [3]:
#Remember that in a huge corpus GTfMatrix will have thousands/millions of documents
#Then docs_num(=21) could be >> 63341
GTfMatrix.shape

(21, 63341)

In [4]:
Ug.shape

(63341, 20)

In [5]:
Ug[gvectorizer.get_feature_names().index('Alice')]

array([-4.40951269e-03,  1.33033600e-02, -4.64223536e-02,  2.42566730e-01,
        9.93370503e-03,  3.05290393e-01,  2.85561843e-01, -4.90058307e-02,
       -2.59280455e-02,  1.05681402e-01, -6.18374582e-02, -1.12536478e-02,
       -7.82374847e-03, -1.27033488e-02,  1.24293473e-02, -5.06080200e-03,
        1.83999617e-03,  2.93662481e-04,  2.10870274e-03,  8.89107309e-05])

## Sklearn Word2Vec-Cosine sentence similarity

### Wrangling Data

From string-sentences to numerical vector representation.

In [17]:
sentence1 = 'the girl run into the hall'
sentence2 = 'Here Alice run to the hall'

In [18]:
import numpy as np

def preproc_data(sentence1, sentence2, Ug):

    sent1 = sentence1.split()
    sent2 = sentence2.split()

    svd_sent1 = []
    svd_sent2 = []

    for word in sent1:
        try:
            svd_sent1.append(Ug[gvectorizer.get_feature_names().index(word)])
        except:
            pass

    for word in sent2:
        try:
            svd_sent2.append(Ug[gvectorizer.get_feature_names().index(word)])
        except:
            pass


    svd_sent1 = sum(np.asarray(svd_sent1))
    svd_sent2 = sum(np.asarray(svd_sent2))
    A = svd_sent1.reshape(1,-1)
    B = svd_sent2.reshape(1,-1)
    
    return A, B

In [19]:
svd_sent1, svd_sent2 = preproc_data(sentence1,sentence2,Ug)
print(len(svd_sent1[0]))
svd_sent1[:10]

20


array([[ 9.27926220e-02, -7.19857997e-02, -1.23356876e-02,
        -2.01910199e-01,  3.93403618e-02,  1.46199465e-01,
         4.58075277e-04, -2.54710224e-02, -7.45007534e-03,
         2.05383102e-01, -4.21420949e-01,  5.08102298e-02,
        -1.93733209e-01, -4.41305574e-01,  1.14012140e-01,
        -1.65396687e-01,  9.80674791e-01, -1.59028786e-02,
        -2.73517252e-01,  1.35984384e+00]])

### Applying Similarity

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(svd_sent1,svd_sent2)[0][0]

0.7380034007075583

In [21]:
#Filtering stopwords
sent1s = 'girl run hall'
sent2s = 'Alice run hall'
svd_sent1s, svd_sent2s = preproc_data(sent1s,sent2s,Ug)
cosine_similarity(svd_sent1s,svd_sent2s)[0][0]

-0.22301475770675458

In [22]:
from scipy.spatial.distance import cosine as cosine_scipy

print(cosine_scipy(svd_sent1,svd_sent2))
print(cosine_scipy(svd_sent1s,svd_sent2s)) #Filtering stopwords

0.26199659929244157
1.2230147577067545


## Best Pair Word Overlap Similarity

Lets try a different way to compound a sentence similarity, based on WordNet-Augmented-Word-Overlap similarity idea.

$p = {\sum_{w\in\ sent_1}max(df[w][w']) \over len(sent_1)} \ \ \ \forall\ w' \in\ sent_2$

$q = {\sum_{w'\in\ sent_2}max(df[w][w']) \over len(sent_2)} \ \ \ \forall\ w \in\ sent_1$

$sim = \left\{ \begin{array}{rcl} 
0  & if\ p+q = 0\\
{2 p*q \over (p+q)}  & others\\
\end{array}
\right.$

In [28]:
def svd_sim(word1,word2, Ug, gvectorizer):
    try:
        a = Ug[gvectorizer.get_feature_names().index(word1)]
        b = Ug[gvectorizer.get_feature_names().index(word2)]
        a = a.reshape(1,-1)
        b = b.reshape(1,-1)
        return cosine_similarity(a,b)[0][0]
    except:
        return 0.0

print(svd_sim('wife','husband', Ug, gvectorizer))

0.582328641082437


In [29]:
svd_sim('car','vehicle', Ug, gvectorizer)

0.3712829259505216

In [30]:
def harmonic_best_pair_word_sim(sentence1,sentence2, Ug, gvectorizer):
    
    sent1 = sentence1.split()
    sent2 = sentence2.split()
    
    p=0
    for wi in sent1:
        m = 0
        for wc in sent2:
            try:
                m = max(m, svd_sim(wi,wc, Ug, gvectorizer))
            except:
                pass
        p += m
    p = p/len(sent1)

    q=0
    for wc in sent2:
        m = 0
        for wi in sent1:
            try:
                m = max(m, svd_sim(wi,wc, Ug, gvectorizer))
            except:
                pass
        q += m
    q = q/len(sent2)

    sim = 2*p*q/(p+q or 1)
    return sim

In [31]:
print('Harmonic mean best pair-word similarity, stopword_filtering=no',
      harmonic_best_pair_word_sim(sentence1,sentence2, Ug, gvectorizer))
print('Same sentences harmonic best word similarity, stopword_filtering=no',
      harmonic_best_pair_word_sim(sentence1,sentence1, Ug, gvectorizer))
print('Harmonic mean best pair-word similarity, stopword_filtering=yes',
      harmonic_best_pair_word_sim(sent1s,sent2s, Ug, gvectorizer))

Harmonic mean best pair-word similarity, stopword_filtering=no 0.7383742690439515
Harmonic mean best pair-word similarity, stopword_filtering=no 1.0
Harmonic mean best pair-word similarity, stopword_filtering=yes 0.7924553499961652


# Wighting TfIdf model with Okapi BM25

This method join with matrix factorization have been showing good results in Information Retrieval, the next cells are a basic experiment to make an initial presentation of future works.

## Applying BM25

The next implementation appears on [Ben Frederickson article](http://www.benfrederickson.com/matrix-factorization/), the complete code of his implementation can be found on [benfrederickson.github](https://github.com/benfred/implicit).

In [32]:
import numpy
from numpy import log, bincount

def bm25_weight(X, K1=100, B=0.8):
    """ Weighs each row of a sparse matrix X  by BM25 weighting """
    # calculate idf per term (user)
    X = coo_matrix(X)

    N = float(X.shape[0])
    idf = log(N / (1 + bincount(X.col)))

    # calculate length_norm per document (artist)
    row_sums = numpy.ravel(X.sum(axis=1))
    average_length = row_sums.mean()
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    X.data = X.data * (K1 + 1.0) / (K1 * length_norm[X.row] + X.data) * idf[X.col]
    return X

In [33]:
WGTfMatrix = bm25_weight(coGTfMatrix)
Ugt, Sgt, Vgt = scipy.sparse.linalg.svds(WGTfMatrix, k=20)

In [34]:
import numpy as np

def svd_sentence_similarity(sentence1,sentence2,Ugt,gvectorizer):
    
    sent1 = sentence1.split()
    sent2 = sentence2.split()

    svd_sent1 = []
    svd_sent2 = []

    for word in sent1:
        try:
            svd_sent1.append(Ugt[gvectorizer.get_feature_names().index(word)])
        except:
            pass

    for word in sent2:
        try:
            svd_sent2.append(Ugt[gvectorizer.get_feature_names().index(word)])
        except:
            pass

    svd_sent1 = sum(np.asarray(svd_sent1))
    svd_sent2 = sum(np.asarray(svd_sent2))
    
    A = svd_sent1.reshape(1,-1)
    B = svd_sent2.reshape(1,-1)
    
    return cosine_similarity(A,B)[0][0]

In [35]:
print(svd_sentence_similarity(sentence1,sentence2,Ug,gvectorizer))
print(svd_sentence_similarity(sent1s,sent2s,Ug,gvectorizer))
print(svd_sentence_similarity(sentence1,sentence2,Ugt,gvectorizer))
print(svd_sentence_similarity(sent1s,sent2s,Ugt,gvectorizer))

0.7380034007075583
-0.22301475770675458
0.5366959906888554
0.43171833850443275


In [38]:
sentence3 = 'boy eat red apple'
print(svd_sentence_similarity(sentence1,sentence3,Ugt,gvectorizer))
print(svd_sentence_similarity(sentence1,sentence3,Ug,gvectorizer))

0.7655415866709938
-0.008266910259872418


In [41]:
svd_sent2.shape

(1, 20)

In [45]:
from gensim.matutils import kullback_leibler, hellinger

print(hellinger(svd_sent1,svd_sent2))
print(kullback_leibler(svd_sent1, svd_sent2))

nan
inf


/home/abelm/flask/lib/python3.5/site-packages/gensim/matutils.py:538: RuntimeWarning: invalid value encountered in sqrt
  sim = np.sqrt(0.5 * ((np.sqrt(vec1) - np.sqrt(vec2))**2).sum())


# Conclusions

* As you can test the SVD+Tf doesn't have a fast or parallel solution, but is a fast method.
* After some mathematical operations (SVD, Transpose, etc) the word-features matrix is obtained. 
* There are no variations between scores of different libraries or methods.
* The application of BM25 do not improve the results in this set of sentences.

# Recomendations

* Made the same example with Wikipedia dump data, to test the similarity difference according to data.

# Future Works

* svd_sentence_similarity get confuse results.